<a id="TableOfContents"></a>
# TABLE OF CONTENTS:
<li><a href='#imports'>Imports</a></li>
<li><a href="#telco">Get Telco Dataset</a></li>
<li><a href="#split">Split Telco Dataset</a></li>
<li><a href="#keycol">Key Columns</a></li>
<li><a href="#DTC">Decision Tree Classifier Modeling</a></li>
<li><a href="#RFC">Random Forest Classifier Modeling</a></li>
<li><a href="#KNN">K-Nearest Neighbors Modeling</a></li>
<li><a href="#LR">Logistic Regression Modeling</a></li>
<li><a href="#top3">Top 3 Models</a></li>

##### Orientation:
The purpose of this file is to create models to predict whether or not a customer will churn and get the best performing models.

<a id='imports'></a>
# IMPORTS:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [1]:
# tabular data
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# DecisionTree modeling:
from sklearn.tree import DecisionTreeClassifier as DTC, export_text, plot_tree

# RandomForest modeling:
from sklearn.ensemble import RandomForestClassifier as RFC

# KNN modeling:
from sklearn.neighbors import KNeighborsClassifier as KNN

# Logistic Regression modeling:
from sklearn.linear_model import LogisticRegression as LR

# Other sklearn stuff
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# .py files
import prepare
import evaluation
import model

<a id='telco'></a>
# Get Telco Dataset:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [2]:
# Acquire cleaned 'telco' dataset
telco = prepare.prep_telco()
telco.sample()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,contract_type_Two_year,internet_service_type_DSL,internet_service_type_Fiber_optic,internet_service_type_None,payment_type_Bank_transfer_(automatic),payment_type_Credit_card_(automatic),payment_type_Electronic_check,payment_type_Mailed_check,total_services,total_extra_services
6209,Male,0,Yes,Yes,63,Yes,Yes,No internet service,No internet service,No internet service,...,1,0,0,1,0,1,0,0,2,0


<a id='split'></a>
# Split Telco Dataset:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [3]:
# Split telco dataset
train, val, test = prepare.split(telco, 'churn')

train.shape:(3943, 67)
validate.shape:(1691, 67)
test.shape:(1409, 67)


In [4]:
# Verify split shapes
train.shape, val.shape, test.shape

((3943, 67), (1691, 67), (1409, 67))

<a id='keycol'></a>
# Key Columns:
<li><a href='#TableOfContents'>Table of Contents</a></li>

- Columns:
    - 'partner'
    - 'dependents'
    - 'online_security'
    - 'online_backup'
    - 'device_protection'
    - 'tech_support'
    - 'streaming_tv'
    - 'streaming_movies'
    - 'payment_type'
    - 'contract_type'
    - 'total_services'
    - 'total_extra_services'

In [5]:
# List of pertinent columns
keylist = [
    'partner_Yes',
    'partner_No',
    'dependents_Yes',
    'dependents_No',
    'online_security_Yes',
    'online_security_No',
    'online_security_No_internet_service',
    'online_backup_Yes',
    'online_backup_No',
    'online_backup_No_internet_service',
    'device_protection_Yes',
    'device_protection_No',
    'device_protection_No_internet_service',
    'tech_support_Yes',
    'tech_support_No',
    'tech_support_No_internet_service',
    'streaming_tv_Yes',
    'streaming_tv_No',
    'streaming_tv_No_internet_service',
    'streaming_movies_Yes',
    'streaming_movies_No',
    'streaming_movies_No_internet_service',
    'payment_type_Bank_transfer_(automatic)',
    'payment_type_Credit_card_(automatic)',
    'payment_type_Electronic_check',
    'payment_type_Mailed_check',
    'contract_type_Month-to-month',
    'contract_type_One_year',
    'contract_type_Two_year',
    'total_services',
    'total_extra_services'
]

In [6]:
# Assign x/y train/val/test cols:
x_train = train[keylist]
y_train = pd.DataFrame(train['churn'])
x_val = val[keylist]
y_val = val['churn']
x_test = test[keylist]
y_test = test['churn']

<a id='DTC'></a>
# Decision Tree Classifier Modeling:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [7]:
# Make df for Decision Tree Models
dtcmodels = pd.DataFrame(train.churn)

In [8]:
# Create baseline
dtcmodels['baseline'] = 'No'

In [9]:
# Check DTC models
dtcmodels.sample()

,churn,baseline
532,No,No


In [10]:
# Create base dictionary
dtcscores = {
    'model' : ['actual'],
    'train' : [100],
    'val' : [100],
    'diff' : [0],
    'test' : [100]
}

In [11]:
# Add baseline to dictionary
baselinetrain = round((train.churn == 'No').mean(), 5)
baselineval = round((val.churn == 'No').mean(), 5)
baselinediff = round(abs(baselinetrain - baselineval), 5)
baselinetest = round((test.churn == 'No').mean(), 5)
dtcscores['model'].append('baseline')
dtcscores['train'].append(baselinetrain)
dtcscores['val'].append(baselineval)
dtcscores['diff'].append(baselinediff)
dtcscores['test'].append(baselinetest)

In [12]:
# Confirm df funcitonality with actual and baseline
pd.DataFrame.from_dict(dtcscores)

,model,train,val,diff,test
0,actual,100.00000,100.00000,0.00000,100.00000
1,baseline,0.73472,0.73448,0.00024,0.73456


In [13]:
# Make and fit DTC of 2-15 depth to train
# Add predictions to DTCdf
for i in range(2, 16):
    dtc = DTC(max_depth=i, random_state=100)
    dtc.fit(x_train, y_train)
    model = dtc.predict(x_train)
    trainscore = round(dtc.score(x_train, y_train), 5)
    valscore = round(dtc.score(x_val, y_val), 5)
    diffscore = round(abs(trainscore - valscore), 5)
    testscore = round(dtc.score(x_test, y_test), 5)
    dtcmodels[i] = model
    dtcscores['model'].append(f'model{i}')
    dtcscores['train'].append(trainscore)
    dtcscores['val'].append(valscore)
    dtcscores['diff'].append(diffscore)
    dtcscores['test'].append(testscore)

In [14]:
# Find best 'train' model
pd.DataFrame.from_dict(dtcscores).sort_values(by='train', ascending=False).head(2)

,model,train,val,diff,test
0,actual,100.0000,100.00000,0.00000,100.00000
15,model15,0.8808,0.72028,0.16052,0.70688


In [15]:
# Find best 'val' model
pd.DataFrame.from_dict(dtcscores).sort_values(by='val', ascending=False).head(2)

,model,train,val,diff,test
0,actual,100.00000,100.00000,0.00000,100.00000
4,model4,0.78849,0.76818,0.02031,0.76011


In [16]:
# Find best 'test' model
pd.DataFrame.from_dict(dtcscores).sort_values(by='test', ascending=False).head(2)

,model,train,val,diff,test
0,actual,100.00000,100.00000,0.00000,100.00000
4,model4,0.78849,0.76818,0.02031,0.76011


- Best Train Model:
    - model 15 @ 88.1%
        - Restrictions:
            - max_depth = 15
            - random_state=100
- Best Validate Model:
    - model 4 @ 76.8%
        - Restrictions:
            - max_depth = 4
            - random_state=100
- Best Test Model:
    - model 4 @ 76.0%
        - Restrictions:
            - max_depth = 4
            - random_state=100

<a id='RFC'></a>
# Random Forest Classifier Modeling:
<li><a href='#TableOfContents'>Table of Contents</a></li>

<a id='KNN'></a>
# K-Nearest Neighbors Modeling:
<li><a href='#TableOfContents'>Table of Contents</a></li>

<a id='LR'></a>
# Logistic Regression Modeling:
<li><a href='#TableOfContents'>Table of Contents</a></li>

<a id='top3'></a>
# Top 3 Models:
<li><a href='#TableOfContents'>Table of Contents</a></li>